In [5]:
import os, sys
import pandas as pd
import numpy as np

In [21]:
df = pd.read_csv('/home/yuen_chen/ENCO/experiments/out/cancer/prompts_obs200_int3_int-stat1_shuf5_rules_Qwen3-4B-Thinking-2507.csv')


In [24]:
print(df['raw_response'].iloc[0])

 {"variables": ["Pollution", "Smoker", "Cancer", "Xray", "Dyspnoea"], "adjacency_matrix": [...]}

We are given 5 variables: 
  0: Pollution
  1: Smoker
  2: Cancer
  3: Xray
  4: Dyspnoea

We have two types of data: 
  (a) Observational data (100 cases, but note: the problem says 100 cases? Actually, the list goes from 1 to 200, but the problem says "Case 1" to "Case 200" for observational? Actually, the problem states 200 cases for observational? But the interventional data is given as 3 cases per intervention? Let me re-read.

However, note: the problem says "The following cases were observed without any intervention" and then lists 200 cases (from Case 1 to Case 200). Then it says "The following cases were observed after specific interventions" and then lists 3 cases per intervention (for 5 interventions: Smoker=False, Xray=positive, Cancer=True, Dyspnoea=False, Pollution=low).

But note: the interventional data is described as:

  When an intervention was performed to set 'Smoker' 

In [4]:
import json
from collections import defaultdict
from pathlib import Path

# Path to your stats file
stats_path = Path("out/cancer/prompts_obs200_int3_int-stat1_shuf5_anon_stats.jsonl")

def normalize(obj, ndigits=8):
    """
    Recursively:
    - round floats to ndigits
    - sort dict keys
    so that semantically identical regimes compare equal.
    """
    if isinstance(obj, dict):
        return {k: normalize(v, ndigits) for k, v in sorted(obj.items())}
    elif isinstance(obj, list):
        return [normalize(v, ndigits) for v in obj]
    elif isinstance(obj, float):
        return round(obj, ndigits)
    else:
        return obj

# Load all records and group them by data_idx
groups = defaultdict(list)

with stats_path.open("r", encoding="utf-8") as f:
    for line in f:
        line = line.strip()
        if not line:
            continue
        rec = json.loads(line)
        groups[rec["data_idx"]].append(rec)

mismatches = []

for data_idx, records in groups.items():
    if len(records) <= 1:
        continue  # nothing to compare

    # Use first shuffle as reference
    ref_regimes = normalize(records[0]["regimes"])
    ref_shuffle = records[0]["shuffle_idx"]

    for rec in records[1:]:
        cur_regimes = normalize(rec["regimes"])
        cur_shuffle = rec["shuffle_idx"]
        if ref_regimes != cur_regimes:
            mismatches.append((data_idx, ref_shuffle, cur_shuffle))

# Report
if not mismatches:
    print(f"✅ All regimes are consistent across shuffle_idx for each data_idx in {stats_path}")
else:
    print("❌ Found regime mismatches across shuffles:")
    for data_idx, s0, s1 in mismatches[:20]:  # show up to 20
        print(f"  data_idx={data_idx}: shuffle_idx {s0} vs {s1}")
    if len(mismatches) > 20:
        print(f"  ... and {len(mismatches) - 20} more mismatches")


✅ All regimes are consistent across shuffle_idx for each data_idx in out/cancer/prompts_obs200_int3_int-stat1_shuf5_anon_stats.jsonl
